# Experiment Mesh Analysis


In [1]:
# external imports
import pandas as pd
from ast import literal_eval
from shapely import (
    from_geojson
)

In [2]:
# internal imports
from src.utils.common_functions import (
    json_file_to_dict,
    get_list_files_in_path,
)
from src.utils.constants import (
    EEE_COUNTRIES_FILEPATH,
    REPLICATION_PACKAGE_DIR,
    TRAFFIC_LOGS_IP_CLASSIFIED_FILEPATH,
    ANYCAST_PII_TRAFFIC_LOGS_FILEPATH,
    APKS_METADATA_FILEPATH,
    RESULTS_MODES,
    IT_ANNOTATION_FILEPATH,
    TPLS_RESULTS_FILEPATH,
    ANYCAST_IP_CLASSIFICATION_FILEPATH
)

In [3]:
# Constants
EEE_countries_set = set([country["alpha-2"] for country in json_file_to_dict(EEE_COUNTRIES_FILEPATH)])

In [4]:
# Analysis params
DESTINATION_REPETITIONS_LIMIT = 1
ANALYSIS_MODE=RESULTS_MODES[0]
GENERATE_ROUTES_DATA=False

In [5]:
# Filepaths variables
EXPERIMENT_RESULTS_FOLDER = f"{REPLICATION_PACKAGE_DIR}/experiment_results_{ANALYSIS_MODE}"
ANALYSIS_FOLDER = f"{REPLICATION_PACKAGE_DIR}/analysis_{ANALYSIS_MODE}"

ROUTES_RESULTS_FILENAME = f"{ANALYSIS_FOLDER}/routes_results_{ANALYSIS_MODE}.csv"
ROUTES_FREQUENCY_FILENAME = f"{ANALYSIS_FOLDER}/routes_frequency_{ANALYSIS_MODE}.csv"

ANYCAST_PII_TRAFFIC_LOGS_ANALYSIS_FILEPATH = f"{REPLICATION_PACKAGE_DIR}/analysis_{ANALYSIS_MODE}/Anycast_PII_Traffic_Logs_{ANALYSIS_MODE}.csv"
TRAFFIC_LOGS_IP_CLASSIFIED_ANALYSIS_FILEPATH = f"{REPLICATION_PACKAGE_DIR}/analysis_{ANALYSIS_MODE}/Traffic_logs_10K_ip_classified_{ANALYSIS_MODE}.csv"
ANYCAST_PII_TRAFFIC_LOGS_ANALYSIS_AGGREGATION_FILEPATH = f"{REPLICATION_PACKAGE_DIR}/analysis_{ANALYSIS_MODE}/Anycast_PII_Traffic_Logs_aggregation_{ANALYSIS_MODE}.csv"
TRAFFIC_LOGS_IP_CLASSIFIED_ANALYSIS_AGGREGATION_FILEPATH = f"{REPLICATION_PACKAGE_DIR}/analysis_{ANALYSIS_MODE}/Traffic_logs_10K_ip_classified_aggregation_{ANALYSIS_MODE}.csv"

## Enrichment of data and generation of datasets

Auxiliar just if needed

In [6]:
def populate_dataframe_with_ip_classification(dataframe: pd.DataFrame) -> pd.DataFrame:
    ips_classified = json_file_to_dict(ANYCAST_IP_CLASSIFICATION_FILEPATH)

    for ip_classified in ips_classified.keys():
        dataframe.loc[dataframe["ip_dest"] == ip_classified, "ip_anycast"] = ips_classified[ip_classified]
    
    return dataframe    

Generate a file with all the routes got as the result of the experiment execution

In [7]:
# Auxiliary functions
def get_probe_location(probe_id: int, origin_list: []) -> (float, float):
    for origin in origin_list:
        if probe_id == origin["probe_id"]:
            location = from_geojson(origin["location"])
            return location.y, location.x
        else:
            continue
    return 0, 0

def get_result_country_route(hunter_result: dict) -> dict:
    probe_id = hunter_result["origin_id"]
    result_country = hunter_result["location_result"]["country"]
    probe_country = hunter_result["origin_country_code"]

    return {
        "origin_id": probe_id,
        "origin_country": probe_country,
        "result_country": result_country
    }

# Data generation function
def generate_routes_raw():
    routes_raw_df = pd.DataFrame(
        columns=[
            "target", "probe_id", "ips_previous_to_target",
            "origin_country", "origin_latitude", "origin_longitude", 
            "result_country", "result_latitude", "result_longitude",
            "result_filename", "outside_EEE"
        ]
    )
    for result_filename in get_list_files_in_path(EXPERIMENT_RESULTS_FOLDER):
        print(result_filename)
        result = json_file_to_dict(f"{EXPERIMENT_RESULTS_FOLDER}/{result_filename}")
        target = result["target"]
        origin_list = result["measurements"]["origin"]
        for hunter_result in result["hunter_results"]:
            route = get_result_country_route(hunter_result)
            probe_id = hunter_result["origin_id"]
            origin_country = route["origin_country"]
            origin_latitude, origin_longitude = get_probe_location(probe_id, origin_list)
            
            result_country = route["result_country"]
            if len(hunter_result["location_result"]["airports_intersection"]) == 1:
                result_location = from_geojson(hunter_result["location_result"]["airports_intersection"][0]["location"])
                result_latitude = result_location.y
                result_longitude = result_location.x
            else:
                result_latitude = 0
                result_longitude = 0
                
            outside_eee = (result_country not in EEE_countries_set) and (result_country != "Indeterminate")
            
            if result_country == "Indeterminate":
                ips_previous_to_target = ["Indeterminate"]
            else:
                ips_previous_to_target = [
                    ip["ip"]
                    for ip in hunter_result["ips_previous_to_target"]
                ]
            
            routes_raw_df = pd.concat(
                [pd.DataFrame([[
                    target, probe_id, str(ips_previous_to_target),
                    origin_country, origin_latitude, origin_longitude, 
                    result_country, result_latitude, result_longitude,
                    result_filename, outside_eee
                ]], columns=routes_raw_df.columns), routes_raw_df], 
                ignore_index=True
            )
    # Sort and save
    routes_raw_df.sort_values(by=["target", "origin_country", "result_country"], inplace=True)
    routes_raw_df.to_csv(ROUTES_RESULTS_FILENAME, sep=",", index=False)


Aggregate and count routes repetitions

In [8]:
# Data generation function
def generate_routes_frequency_aggregation():
    # Aggregate routes counting the repetitions
    routes_frequency_df = pd.read_csv(ROUTES_RESULTS_FILENAME, sep=",")
    routes_frequency_df = routes_frequency_df[["target", "origin_country", "result_country"]]
    routes_frequency_df = routes_frequency_df.value_counts(subset=['target', 'origin_country', 'result_country'])
    routes_frequency_df.to_csv(ROUTES_FREQUENCY_FILENAME, sep=",")
    
    # Include the info about outside the EEE
    routes_frequency_df = pd.read_csv(ROUTES_FREQUENCY_FILENAME, sep=",")
    routes_frequency_df["outside_EEE"] = False
    
    routes_frequency_df.loc[
        (routes_frequency_df["result_country"] != "Indeterminate") &
        (~routes_frequency_df["result_country"].isin(EEE_countries_set)),
        ["outside_EEE"]
    ] = True
    routes_frequency_df.to_csv(ROUTES_FREQUENCY_FILENAME, sep=",", index=False)


Introduce routes outside EEE and its count in the complete dataset WHAT DATASETS POPULATE????

In [9]:
# Dataset population function
def populate_dataset_with_routes_results(dataframe: pd.DataFrame) -> pd.DataFrame:
    routes_valid_df = pd.read_csv(ROUTES_FREQUENCY_FILENAME, sep=",")
    routes_valid_df = routes_valid_df.loc[
        (routes_valid_df["outside_EEE"] == True) & 
        (routes_valid_df["count"] >= DESTINATION_REPETITIONS_LIMIT)
    ]

    # Charge default routes in the dataset to populate
    dataframe["origins_transfers_outside_EEE"] = "[]"
    dataframe["destinations_transfers_outside_EEE"] = "[]"
    dataframe["frequency_transfers_outside_EEE"] = "[]"
    dataframe["outside_EEE"] = False
    
    # For every IP get the list of origins, destinations and frequency and save it
    for ip in routes_valid_df["target"].unique().tolist():
        ip_routes = routes_valid_df.loc[routes_valid_df["target"] == ip]
        origins_transfers_outside_eee = ip_routes["origin_country"].values.tolist()
        destinations_transfers_outside_eee = ip_routes["result_country"].values.tolist()
        frequency_transfers_outside_eee = ip_routes["count"].values.tolist()

        dataframe.loc[
            (dataframe["ip_dest"] == ip), 
            ["origins_transfers_outside_EEE", 
             "destinations_transfers_outside_EEE", 
             "frequency_transfers_outside_EEE",
             "outside_EEE"]
        ] = [str(origins_transfers_outside_eee),
             str(destinations_transfers_outside_eee),
             str(frequency_transfers_outside_eee),
             True]
    
    return dataframe


Populate the datasets with metadata info

In [10]:
def populate_dataset_with_apks_metadata(dataframe: pd.DataFrame) -> pd.DataFrame:
    apk_metadata_df = pd.read_csv(APKS_METADATA_FILEPATH, sep=",")
    
    return dataframe.merge(
        apk_metadata_df,
        on=["apk", "version"],
        how="left"
    )


Populate the datasets with the info extrated from every policy

In [11]:
def populate_dataset_with_policy_extracted_info(dataframe: pd.DataFrame) -> pd.DataFrame:
    it_annotation_results_df = pd.read_csv(IT_ANNOTATION_FILEPATH, sep=",")
    
    it_annotation_results_df.drop_duplicates(["apk", "countries"], inplace=True)
    it_annotation_results_df.fillna(
        value={"countries": "[]"},
        inplace=True
    )
    it_annotation_results_df.rename(
        columns={
            "transfer": "it_mentioned_by_policy",
            "adequacy_decision": "adequacy_decision_by_policy",
            "countries": "countries_mentioned_by_policy"
        }, 
        inplace=True
    )
    
    dataframe = pd.merge(
        dataframe,
        it_annotation_results_df[[
            "apk", "version", 
            "it_mentioned_by_policy", "adequacy_decision_by_policy", "countries_mentioned_by_policy"
        ]], 
        on=["apk", "version"],
        how="left"
    )

    dataframe.fillna(
        value={
            "countries_mentioned_by_policy": "[]",
            "it_mentioned_by_policy": False,
            "adequacy_decision_by_policy": False
        }, inplace=True
    )
    
    return dataframe
    

Populate the datasets with the info about the libraries which carried out the communication 

In [12]:
def populate_dataset_with_libraries_data(dataframe: pd.DataFrame) -> pd.DataFrame:
    tpls_results_df = pd.read_csv(TPLS_RESULTS_FILEPATH, sep=",")
    
    dataframe = pd.merge(
        dataframe,
        tpls_results_df[[
            "apk", "version", "port_source", "host", "port_dest", "ip_dest",
            "TP-performed", "TP-library", "FP-intended"
        ]],
        on=["apk", "version", "port_source", "host", "port_dest", "ip_dest"],
        how="left"
    )
    
    return dataframe

Check GDPR compliance in terms of international transfers

In [13]:
def check_apk_it_gdpr_compliance(dataframe: pd.DataFrame) -> pd.DataFrame:
    dataframe["apk_it_gdpr_compliance"] = True
    
    dataframe["apk_it_gdpr_compliance"] = dataframe.apply(
        lambda row_to_check: 
        set(literal_eval(row_to_check["destinations_transfers_outside_EEE"])).issubset(
            set(literal_eval(row_to_check["countries_mentioned_by_policy"]))
        ),
        axis=1
    )
    
    return dataframe
        

Execute the enrichment of data and generation

In [14]:
# Charge the dataframes to be used
traffic_logs_ip_classified_analysis_df = pd.read_csv(TRAFFIC_LOGS_IP_CLASSIFIED_FILEPATH, sep=",")
anycast_pii_traffic_logs_df = pd.read_csv(ANYCAST_PII_TRAFFIC_LOGS_FILEPATH, sep=",")
datasets = [traffic_logs_ip_classified_analysis_df, anycast_pii_traffic_logs_df]
analysis_filepaths = [TRAFFIC_LOGS_IP_CLASSIFIED_ANALYSIS_FILEPATH, ANYCAST_PII_TRAFFIC_LOGS_ANALYSIS_FILEPATH]

In [15]:
# Because is a long process and is only necessary to run once I include the condition
if GENERATE_ROUTES_DATA:
    # Generate the results
    generate_routes_raw()
    generate_routes_frequency_aggregation()

In [16]:
# Populate datasets
for index in range(0, len(datasets)):
    dataset_to_improve = datasets[index]
    
    # Populate with routes
    print("Populate with routes")
    dataset_to_improve = populate_dataset_with_routes_results(dataset_to_improve)
    # Populate with metadata
    print("Populate with metadata")
    dataset_to_improve = populate_dataset_with_apks_metadata(dataset_to_improve)
    # Populate with the privacy policy extracted data
    print("Populate with the privacy policy extracted data")
    dataset_to_improve = populate_dataset_with_policy_extracted_info(dataset_to_improve)
    # Populate with libraries data
    print("Populate with the libraries data")
    dataset_to_improve = populate_dataset_with_libraries_data(dataset_to_improve)

    # Check conditions
    print("Checking IT declarations accomplishment")
    check_apk_it_gdpr_compliance(dataset_to_improve)
    
    print("Saving data")
    dataset_to_improve.to_csv(analysis_filepaths[index], sep=",", index=False)
    

Populate with routes
Populate with metadata
Populate with the privacy policy extracted data


/tmp/ipykernel_6564/2843223289.py:28: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataframe.fillna(


Populate with the libraries data
Checking IT declarations accomplishment
Saving data
Populate with routes
Populate with metadata
Populate with the privacy policy extracted data


/tmp/ipykernel_6564/2843223289.py:28: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataframe.fillna(


Populate with the libraries data
Checking IT declarations accomplishment
Saving data


## Analysis questions

Answers to the questions needed for the article

Acronyms:
- PII = Personal Identificable Information

In [17]:
# Data load
traffic_logs_ip_classified_analysis_df = pd.read_csv(TRAFFIC_LOGS_IP_CLASSIFIED_ANALYSIS_FILEPATH, sep=",")
anycast_pii_traffic_logs_analysis_df = pd.read_csv(ANYCAST_PII_TRAFFIC_LOGS_ANALYSIS_FILEPATH, sep=",")

/tmp/ipykernel_6564/3233838539.py:2: DtypeWarning: Columns (6,7,9,17,19,24,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  traffic_logs_ip_classified_analysis_df = pd.read_csv(TRAFFIC_LOGS_IP_CLASSIFIED_ANALYSIS_FILEPATH, sep=",")
/tmp/ipykernel_6564/3233838539.py:3: DtypeWarning: Columns (36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  anycast_pii_traffic_logs_analysis_df = pd.read_csv(ANYCAST_PII_TRAFFIC_LOGS_ANALYSIS_FILEPATH, sep=",")


**IPs analysis**

In [18]:
ips_total = traffic_logs_ip_classified_analysis_df["ip_dest"].unique().tolist()
print(f"Number of IPs in traffic logs: {len(ips_total)}")

ips_total_pii = traffic_logs_ip_classified_analysis_df.loc[
    (traffic_logs_ip_classified_analysis_df["PII"] != "No-PII") &
    (traffic_logs_ip_classified_analysis_df["PII"].notna())
    ]["ip_dest"].unique().tolist()
print(f"Number of IPs with PII: {len(ips_total_pii)}")

ips_anycast = traffic_logs_ip_classified_analysis_df.loc[
    traffic_logs_ip_classified_analysis_df["ip_anycast"]
]["ip_dest"].unique().tolist()
print(f"Number of IPs anycast: {len(ips_anycast)}")

ips_anycast_pii = traffic_logs_ip_classified_analysis_df.loc[
    (traffic_logs_ip_classified_analysis_df["ip_anycast"]) &
    (traffic_logs_ip_classified_analysis_df["PII"] != "No-PII") &
    (traffic_logs_ip_classified_analysis_df["PII"].notna())
    ]["ip_dest"].unique().tolist()
print(f"Number of IPs anycast with PII: {len(ips_anycast_pii)}")

Number of IPs in traffic logs: 5647
Number of IPs with PII: 1807
Number of IPs anycast: 991
Number of IPs anycast with PII: 200


**APKS analysis**

In [19]:
apks_total = traffic_logs_ip_classified_analysis_df["apk"].unique().tolist()
print(f"Number of APKs in traffic logs: {len(apks_total)}")

apks_total_pii = traffic_logs_ip_classified_analysis_df.loc[
    (traffic_logs_ip_classified_analysis_df["PII"] != "No-PII") &
    (traffic_logs_ip_classified_analysis_df["PII"].notna())
    ]["apk"].unique().tolist()
print(f"Number of APKs with PII: {len(apks_total_pii)}")

apks_anycast = traffic_logs_ip_classified_analysis_df.loc[
    traffic_logs_ip_classified_analysis_df["ip_anycast"]
]["apk"].unique().tolist()
print(f"Number of APKs using anycast: {len(apks_anycast)}")

apks_anycast_pii = traffic_logs_ip_classified_analysis_df.loc[
    (traffic_logs_ip_classified_analysis_df["ip_anycast"]) &
    (traffic_logs_ip_classified_analysis_df["PII"] != "No-PII") &
    (traffic_logs_ip_classified_analysis_df["PII"].notna())
    ]["apk"].unique().tolist()
print(f"Number of APKs anycast with PII: {len(apks_anycast_pii)}")

Number of APKs in traffic logs: 5759
Number of APKs with PII: 3478
Number of APKs using anycast: 1669
Number of APKs anycast with PII: 960


**Hosts Analysis**

In [20]:
hosts_total = traffic_logs_ip_classified_analysis_df["host"].unique().tolist()
print(f"Number of hosts in traffic logs: {len(hosts_total)}")

hosts_total_pii = traffic_logs_ip_classified_analysis_df.loc[
    (traffic_logs_ip_classified_analysis_df["PII"] != "No-PII") &
    (traffic_logs_ip_classified_analysis_df["PII"].notna())
    ]["host"].unique().tolist()
print(f"Number of hosts with PII: {len(hosts_total_pii)}")

hosts_anycast = traffic_logs_ip_classified_analysis_df.loc[
    traffic_logs_ip_classified_analysis_df["ip_anycast"]
]["host"].unique().tolist()
print(f"Number of hosts using anycast: {len(hosts_anycast)}")

hosts_anycast_pii = traffic_logs_ip_classified_analysis_df.loc[
    (traffic_logs_ip_classified_analysis_df["ip_anycast"]) &
    (traffic_logs_ip_classified_analysis_df["PII"] != "No-PII") &
    (traffic_logs_ip_classified_analysis_df["PII"].notna())
    ]["host"].unique().tolist()
print(f"Number of hosts anycast with PII: {len(hosts_anycast_pii)}")

Number of hosts in traffic logs: 4738
Number of hosts with PII: 966
Number of hosts using anycast: 995
Number of hosts anycast with PII: 201


**Data Types**  

In [21]:
pii_data_types_anycast = anycast_pii_traffic_logs_analysis_df["PII"].unique().tolist()
print(f"Types of PII data treated by anycast IPs:")
print(pii_data_types_anycast)

pii_data_types_anycast_pii_it = anycast_pii_traffic_logs_analysis_df.loc[
    anycast_pii_traffic_logs_analysis_df["outside_EEE"]
]["PII"].unique().tolist()
print(f"Types of PII data treated by anycast IPs that make IT:")
print(pii_data_types_anycast_pii_it)

Types of PII data treated by anycast IPs:
['Device_Model', 'Google_Ad_ID', 'Build_No', 'Kernel_version', 'Fingerprint', 'Router_Wifi_BSSID', 'Router_Wifi_BSSID_Close', 'Router_Wifi_MAC', 'Device_location', 'Device_location_coarse']
Types of PII data treated by anycast IPs that make IT:
['Device_Model', 'Google_Ad_ID', 'Build_No', 'Kernel_version', 'Fingerprint', 'Router_Wifi_BSSID', 'Router_Wifi_BSSID_Close', 'Router_Wifi_MAC', 'Device_location', 'Device_location_coarse']


**GDPR Compliance**

In [22]:
apks_anycast_pii_declare_it = anycast_pii_traffic_logs_analysis_df.loc[
    anycast_pii_traffic_logs_analysis_df["it_mentioned_by_policy"]
]["apk"].unique().tolist()
print(f"Number of APKs that use anycast IPs and treat PII that declare IT in privacy policy: {len(apks_anycast_pii_declare_it)}")

apks_anycast_pii_not_compliance = anycast_pii_traffic_logs_analysis_df.loc[
    anycast_pii_traffic_logs_analysis_df["apk_it_gdpr_compliance"] == False
]["apk"].unique().tolist()
print(f"Number of APKs that use anycast IPs and treat PII that has not compliance: {len(apks_anycast_pii_not_compliance)}")

apks_anycast_pii_not_compliance_declare_it = anycast_pii_traffic_logs_analysis_df.loc[
    (anycast_pii_traffic_logs_analysis_df["it_mentioned_by_policy"] == True) &
    (anycast_pii_traffic_logs_analysis_df["apk_it_gdpr_compliance"] == False)
    ]["apk"].unique().tolist()
print(f"Number of APKs that use anycast IPs and treat PII that has not compliance and declare IT in privacy policy: {len(apks_anycast_pii_not_compliance_declare_it)}")

Number of APKs that use anycast IPs and treat PII that declare IT in privacy policy: 196
Number of APKs that use anycast IPs and treat PII that has not compliance: 947
Number of APKs that use anycast IPs and treat PII that has not compliance and declare IT in privacy policy: 189


**TLPs**

In [50]:
anycast_pii_it_connections = len(
    anycast_pii_traffic_logs_analysis_df.loc[
        (anycast_pii_traffic_logs_analysis_df["outside_EEE"])
    ].index)
print(f"Number of connections Anycast+PII+IT: {anycast_pii_it_connections}")

anycast_pii_it_connections_unknown = len(
    anycast_pii_traffic_logs_analysis_df.loc[
        (anycast_pii_traffic_logs_analysis_df["outside_EEE"]) &
        (anycast_pii_traffic_logs_analysis_df["TP-performed"] == False) &
        (anycast_pii_traffic_logs_analysis_df["FP-intended"] == False) &
        (anycast_pii_traffic_logs_analysis_df["TP-library"] is None)
    ].index)
print(f"Number of connections Anycast+PII+IT unknow: {anycast_pii_it_connections_unknown}")

anycast_pii_it_connections_by_app = len(
    anycast_pii_traffic_logs_analysis_df.loc[
        (anycast_pii_traffic_logs_analysis_df["outside_EEE"]) &
        (anycast_pii_traffic_logs_analysis_df["TP-performed"] == False) &
        (anycast_pii_traffic_logs_analysis_df["FP-intended"])
    ].index)
print(f"Number of connections Anycast+PII+IT set up by APP: {anycast_pii_it_connections_by_app}")



Number of connections Anycast+PII+IT: 195432
Number of connections Anycast+PII+IT unknow: 0
Number of connections Anycast+PII+IT set up by APP: 89103


In [51]:

anycast_pii_it_connections_by_tpl = len(
    anycast_pii_traffic_logs_analysis_df.loc[
        (anycast_pii_traffic_logs_analysis_df["outside_EEE"] == True) &
        (anycast_pii_traffic_logs_analysis_df["TP-performed"])
        ].index)
print(f"Number of connections Anycast+PII+IT set up by TPL: {anycast_pii_it_connections_by_tpl}")

anycast_pii_it_connections_by_tpl_fp_intended = len(
    anycast_pii_traffic_logs_analysis_df.loc[
        (anycast_pii_traffic_logs_analysis_df["outside_EEE"]) &
        (anycast_pii_traffic_logs_analysis_df["TP-performed"]) &
        (anycast_pii_traffic_logs_analysis_df["FP-intended"])
        ].index)
print(f"Number of connections Anycast+PII+IT set up by TPL but performed from FP: {anycast_pii_it_connections_by_tpl_fp_intended}")


Number of connections Anycast+PII+IT set up by TPL: 83833
Number of connections Anycast+PII+IT set up by TPL but performed from FP: 77873


# STOP

In [24]:
anycast_pii_it_connections_by_app = len(anycast_pii_traffic_logs_analysis_df.loc[
    (anycast_pii_traffic_logs_analysis_df["apk"] == "it.Gruner.Focus") &
    (anycast_pii_traffic_logs_analysis_df["version"] == 100025)
    ].index)
anycast_pii_it_connections_by_app

1

In [25]:
import pandas as pd
import numpy as np

# Crear datos aleatorios para el DataFrame
data_total = {
    'numeros': np.random.randint(0, 100, 10),
    'decimales': np.random.randn(10),
    'letras': np.random.choice(['A', 'B'], 10),
    'booleanos': np.random.choice([True, False], 10)
}

data_spec = {
    'decimales': np.random.randn(10),
    'letras': np.random.choice(['A', 'B'], 10),
    'booleanos': np.random.choice([True, False], 10)
}

# Crear el DataFrame
df_total = pd.DataFrame(data_total)
df_spec = pd.DataFrame(data_spec)


In [26]:
print(df_total)

   numeros  decimales letras  booleanos
0       19   0.853894      A      False
1       42  -0.480985      B      False
2       26  -0.313592      A      False
3        3   0.095294      A       True
4       42   0.598232      A       True
5       52   0.300813      B      False
6       65   1.270196      B       True
7       61   1.216834      B       True
8       13  -1.122564      A      False
9       14   0.891056      A      False


In [27]:
print(df_spec)

   decimales letras  booleanos
0   1.449173      B       True
1  -0.076420      A      False
2   0.660569      A      False
3   1.108656      B       True
4   0.963982      B      False
5  -1.284315      A       True
6  -0.781889      A       True
7   1.430619      A      False
8  -0.576043      B      False
9   0.190054      B      False


In [28]:
def function_test(df):
    return df.merge(df_spec[["letras", "booleanos"]], on=["letras"], how='left').copy()

In [29]:
df_total = function_test(df_total)
df_total


,numeros,decimales,letras,booleanos_x,booleanos_y
0,19,0.853894,A,False,False
1,19,0.853894,A,False,False
2,19,0.853894,A,False,True
3,19,0.853894,A,False,True
4,19,0.853894,A,False,False
5,42,-0.480985,B,False,True
6,42,-0.480985,B,False,True
7,42,-0.480985,B,False,False
8,42,-0.480985,B,False,False
9,42,-0.480985,B,False,False
